In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
from google.colab import drive
drive.mount('/content/drive')
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
your_module = drive.CreateFile({'id':'174Ws0LnaEQdK7UmOKh-8GJwftO18Mfej'})
your_module.GetContentFile('metric.py')
your_module1 = drive.CreateFile({'id':'11ssSCDodnZ6_jXcHLZ8AqYHbg7g7K8Be'})
your_module1.GetContentFile('threshold.py')
your_module2 = drive.CreateFile({'id':'1JpuTOGEKMq9iZcO6Tp4ET9icd5EG0P0D'})
your_module2.GetContentFile('cat_list.py')
your_module2 = drive.CreateFile({'id':'1gsMrGL0BfVASQICf-eesMRelmCgMLTYs'})
your_module2.GetContentFile('cat_list_show.py')
from metric import *
from threshold import *
from cat_list import *
from cat_list_show import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def jacard(y_test, predictions):
    accuracy = 0.0

    for i in range(y_test.shape[0]):
        intersection = 0.0
        union = 0.0
        for j in range(y_test.shape[1]):
            if int(y_test[i,j]) == 1 or int(predictions[i,j]) == 1:
                union += 1
            if int(y_test[i,j]) == 1 and int(predictions[i,j]) == 1:
                intersection += 1
            if int(y_test[i,j]) == 2 or int(predictions[i,j]) == 2:
                union += 1
            if int(y_test[i,j]) == 2 and int(predictions[i,j]) == 2:
                intersection += 1
            
        if union != 0:
            accuracy = accuracy + float(intersection/union)

    accuracy = float(accuracy/y_test.shape[0])

    return accuracy

In [3]:
!pip install tensorflow-determinism

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for tensorflow-determinism: filename=tensorflow_determinism-0.3.0-py3-none-any.whl size=9158 sha256=cb64da250695b1e27b0ce60f7d7b6a74dd120d412e7aac196c721f73cdb2b1bc
  Stored in directory: /root/.cache/pip/wheels/d2/be/33/2b27e81e5d40b4bfb7c103ac6c6c5e81fdbcf40d2af5078529
Successfully built tensorflow-determinism


In [22]:
#تکرار پذیری مدل
import os
sd = 1024
os.environ['PYTHONHASHSEED']=str(sd)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import tensorflow as tf
import numpy as np
import random
import random as rn
random.seed(sd)
np.random.seed(sd)
rn.seed(sd)
from keras import backend as K

initializer = tf.random_normal_initializer(0., 0.02, seed=sd)
#فراخوانی کتابخانه ها
import pandas as pd
from tensorflow.python.keras.models import Model,Sequential
from tensorflow.python.keras.layers import Input,Dense, GRU, Embedding, LSTM,Conv1D
from tensorflow.keras.optimizers import Adam,Adagrad,Nadam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dropout,GlobalMaxPooling1D,Conv1D,BatchNormalization
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras import initializers, regularizers, constraints
from sklearn.metrics import classification_report
from time import time
from tensorflow.keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir='drive/My Drive/logs/{}'.format(time()))

#فراخوانی داده های آموزشی و آزمایشی
dataset = pd.read_excel('/content/drive/MyDrive/تحلیل احساسات سطح ویژگی/train.xlsx')
test_file = pd.read_excel('/content/drive/MyDrive/تحلیل احساسات سطح ویژگی/test.xlsx')
S_comments_labels = dataset[['بازیگر','بازیگری','داستان',
                              'سبک','فیلم',
                             'فیلم_نامه',
                             'محتوا',
                            'موضوع','کارگردان',
                            ]]

S_test_comments_labels = test_file[['بازیگر','بازیگری','داستان',
                              'سبک','فیلم',
                             'فیلم_نامه',
                             'محتوا',
                            'موضوع','کارگردان',
                            ]]
category_list=['بازیگر','بازیگری','داستان',
                              'سبک','فیلم',
                             'فیلم_نامه',
                             'محتوا',
                            'موضوع','کارگردان',
                            ]
stop_words =['به','شکل','ممکن','میشه','دلیل','رو','فقط','در','خانم','آقای','اگر','مریلا',
            'این','هم','اسامی','زارعی','بهرام','رادان','حیایی','پارسا','پیروزفر','و','صابر','ابر','حال','های',
            'ها','یه','ای','بابت','آبیار','از','تا','هفتاد','درصد','الناز','دو','اول','شاکردوست','سیمرغ','زن','تاریخ','شاکر','هاش','ریگی','اش','معادی','نرگس','دوم','ابیار','پیمان','اقای','بهداد','مهناز','حامد','عطاران','علی','نوید','ام','امیر','۱۰','محمدزاده','نفر','تیکت','سعید','لباس','خدا','هانیه'
             ,'توسلی','سحر','۲۰','ثقفی','رضا','کریمی','روسی','۴','کامنت','عبدالمالک','ش','اکبر','هیس','مهرجویی','طباطبایی','سرخپوست','عبدی','شش','شیطان','پرستی'
            ]
#پیش پردازش
import re
def preprocess_text(sentence):
    # Removing multiple spaces
    sentence = re.sub(r'@\S+', '', sentence)
    sentence = re.sub(r'!\S+', '', sentence)
    sentence = re.sub(r'؟\S+', '', sentence)
    sentence = re.sub(r'[.]', ' ', sentence)
    sentence = re.sub(r'[/]', '', sentence)
    sentence = re.sub(r'[،]', ' ', sentence)
    sentence = re.sub(r'[؛]', '', sentence)
    sentence = sentence.split()
    sentence =[word for word in sentence if word not in stop_words]
    sentence = ' '.join(sentence)
    return sentence
def ReplacetwoMore(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL) 
    return pattern.sub(r"\1", s)
dataset['clean_text'] = dataset['Sentence'].apply(ReplacetwoMore)
test_file['clean_text'] = test_file['Sentence'].apply(ReplacetwoMore)
X = []
sentences = list(dataset["clean_text"])
for sen in sentences:
    X.append(preprocess_text(sen))    
X_t = []
sentences2 = list(test_file["clean_text"])
for sen in sentences2:
    X_t.append(preprocess_text(sen))
np.random.seed(sd)

y = S_comments_labels.values
y_test1=S_test_comments_labels.values
target =y
data = X
target1 = y_test1
data1 = X_t
x_train, x_test = data, data1
y_train, y_test = target, target1
np.random.seed(sd)
print(x_train[136])
print(y_train[136])
y1_train = dataset[["بازیگر"]].values
y1_test =  test_file[["بازیگر"]].values
y2_train = dataset[["بازیگری"]].values
y2_test =  test_file[["بازیگری"]].values
y3_train = dataset[["داستان"]].values
y3_test =  test_file[["داستان"]].values

y4_train = dataset[["سبک"]].values
y4_test =  test_file[["سبک"]].values
y5_train = dataset[["فیلم"]].values
y5_test =  test_file[["فیلم"]].values
y6_train = dataset[["فیلم_نامه"]].values
y6_test =  test_file[["فیلم_نامه"]].values


y7_train = dataset[["محتوا"]].values
y7_test =  test_file[["محتوا"]].values

y8_train = dataset[["موضوع"]].values
y8_test =  test_file[["موضوع"]].values
y9_train = dataset[["کارگردان"]].values
y9_test =  test_file[["کارگردان"]].values

#شمارش تعداد کلمات منحصر به فرد
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import FreqDist
all_words=' '.join(data)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
print ('number unique word:',num_unique_word)
#شمارش بزرگترین طول متن
r_len=[]
for text in data:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)   
MAX_REVIEW_LEN=np.max(r_len)
print('max len:',MAX_REVIEW_LEN)
np.random.seed(sd)
num_words = num_unique_word
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data)
np.random.seed(sd)
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)
print(x_train[100])
print(x_train_tokens[100])
max_tokens = MAX_REVIEW_LEN
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,padding='post')
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,padding='post')
np.random.seed(sd)
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text
print('train shape',x_train_pad.shape)
print('test shape',x_test_pad.shape)
np.random.seed(sd)
#طراحی مدل
embedding_size = 300
input_1 = Input(shape=(max_tokens,))
x=Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    name='embedding_layer',
            embeddings_initializer=initializer)(input_1)
x=Dropout(0.2,seed=sd)(x)
x=Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1 ,kernel_initializer=initializer,use_bias=False)(x)
x=BatchNormalization()(x)
x=GlobalMaxPooling1D()(x)

x=Dense(200, activation='relu',
        kernel_initializer=initializer,use_bias=False)(x)
x=Dropout(0.2,seed=sd)(x)
output1 = Dense(3, activation='softmax',name='bazigar')(x)
output2 = Dense(3, activation='softmax',name='bazigari')(x)
output3 = Dense(3, activation='softmax',name='dastan')(x)
output4 = Dense(3, activation='softmax',name='dialog')(x)
output5 = Dense(3, activation='softmax')(x)
output6 = Dense(3, activation='softmax')(x)
output7 = Dense(3, activation='softmax')(x)
output8 = Dense(3, activation='softmax')(x)
output9 = Dense(3, activation='softmax')(x)
model1 = Model(inputs=input_1, outputs=[output1, output2, output3, output4, output5, output6,output7,output8,output9
                                      ])

optimizer =Nadam(learning_rate=1e-3)
model1.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,)
model1.summary()
np.random.seed(sd)
history=model1.fit(x_train_pad, 
                   y=[y1_train,y2_train,y3_train,y4_train,
                                  y5_train,y6_train,y7_train,
                                  y8_train,y9_train]
                   , epochs=20, batch_size=256,shuffle=False,callbacks=[tensorboard])

predicted = model1.predict(x_test_pad)
pred_class = np.argmax(predicted, axis=-1) 
from sklearn.metrics import hamming_loss, accuracy_score,precision_score
p=pred_class.T
hm=[]
ac=[]
x1=[]
x2=[]
for i in range(len(y_test)):
  hm.append(hamming_loss(y_test[i], p[i]))
print(np.mean(hm))
print (jacard(y_test, p))

یک کلام عالی همه چیز فیلم نامه فیلم برداری کارگردان بازیگران موضوع متفاوت فوقالعاده بود یک کلام
[1 0 0 0 0 1 0 1 1]
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
number unique word: 4730
max len: 103
عالی فیلم دارای سر ته مشخص جمع جوری داشت منون کلیه عوامل فیلم
[6, 1, 705, 62, 153, 354, 787, 642, 23, 322, 1280, 123, 1]
train shape (2000, 103)
test shape (200, 103)
Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 103)]        0                                            
__________________________________________________________________________________________________
embedding_layer (Embedding)     (None, 103, 300)     1419000     input_11[0][0]                   
________________________________________________________

In [18]:
embedding_size = 200
input_1 = Input(shape=(max_tokens,))
x=Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    name='embedding_layer',
            embeddings_initializer=initializer)(input_1)
x=Dropout(0.2,seed=sd)(x)
x=Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1 ,kernel_initializer=initializer,use_bias=False)(x)
x=BatchNormalization()(x)
x=GlobalMaxPooling1D()(x)

x=Dense(200, activation='relu',
        kernel_initializer=initializer,use_bias=False)(x)
x=Dropout(0.2,seed=sd)(x)
output1 = Dense(3, activation='softmax',name='bazigar')(x)
output2 = Dense(3, activation='softmax',name='bazigari')(x)
output3 = Dense(3, activation='softmax',name='dastan')(x)
output4 = Dense(3, activation='softmax',name='dialog')(x)
output5 = Dense(3, activation='softmax')(x)
output6 = Dense(3, activation='softmax')(x)
output7 = Dense(3, activation='softmax')(x)
output8 = Dense(3, activation='softmax')(x)
output9 = Dense(3, activation='softmax')(x)
model = Model(inputs=input_1, outputs=[output1, output2, output3, output4, output5, output6,output7,output8,output9
                                      ])

optimizer =Nadam(learning_rate=1e-3)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,)
model.summary()
np.random.seed(sd)
history=model.fit(x_train_pad, 
                   y=[y1_train,y2_train,y3_train,y4_train,
                                  y5_train,y6_train,y7_train,
                                  y8_train,y9_train]
                   , epochs=20, batch_size=256,shuffle=False,callbacks=[tensorboard])

predicted2 = model.predict(x_test_pad)
pred_class = np.argmax(predicted2, axis=-1) 
p2=pred_class.T
hm=[]
ac=[]
x1=[]
x2=[]
for i in range(len(y_test)):
  hm.append(hamming_loss(y_test[i], p2[i]))
print(np.mean(hm))
print (jacard(y_test, p2))

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 103)]        0                                            
__________________________________________________________________________________________________
embedding_layer (Embedding)     (None, 103, 200)     946000      input_8[0][0]                    
__________________________________________________________________________________________________
dropout_13 (Dropout)            (None, 103, 200)     0           embedding_layer[0][0]            
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 103, 256)     153600      dropout_13[0][0]                 
____________________________________________________________________________________________

In [142]:
embedding_size = 100
input_1 = Input(shape=(max_tokens,))
x=Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    name='embedding_layer',
            embeddings_initializer=initializer)(input_1)
x=Dropout(0.2,seed=sd)(x)
x=Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1 ,kernel_initializer=initializer,use_bias=False)(x)
x=BatchNormalization()(x)
x=GlobalMaxPooling1D()(x)

x=Dense(200, activation='relu',
        kernel_initializer=initializer,use_bias=False)(x)
x=Dropout(0.2,seed=sd)(x)
output1 = Dense(3, activation='softmax',name='bazigar')(x)
output2 = Dense(3, activation='softmax',name='bazigari')(x)
output3 = Dense(3, activation='softmax',name='dastan')(x)
output4 = Dense(3, activation='softmax',name='dialog')(x)
output5 = Dense(3, activation='softmax')(x)
output6 = Dense(3, activation='softmax')(x)
output7 = Dense(3, activation='softmax')(x)
output8 = Dense(3, activation='softmax')(x)
output9 = Dense(3, activation='softmax')(x)
model3 = Model(inputs=input_1, outputs=[output1, output2, output3, output4, output5, output6,output7,output8,output9
                                      ])

optimizer =Nadam(learning_rate=1e-3)
model3.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,)
model3.summary()
np.random.seed(sd)
history=model3.fit(x_train_pad, 
                   y=[y1_train,y2_train,y3_train,y4_train,
                                  y5_train,y6_train,y7_train,
                                  y8_train,y9_train]
                   , epochs=20, batch_size=256,shuffle=False,callbacks=[tensorboard])

predicted3 = model3.predict(x_test_pad)
pred_class = np.argmax(predicted3, axis=-1) 
p3=pred_class.T
hm=[]
ac=[]
x1=[]
x2=[]
for i in range(len(y_test)):
  hm.append(hamming_loss(y_test[i], p3[i]))
print(np.mean(hm))
print (jacard(y_test, p3))

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 103)]        0                                            
__________________________________________________________________________________________________
embedding_layer (Embedding)     (None, 103, 100)     473000      input_14[0][0]                   
__________________________________________________________________________________________________
dropout_25 (Dropout)            (None, 103, 100)     0           embedding_layer[0][0]            
__________________________________________________________________________________________________
conv1d_12 (Conv1D)              (None, 103, 256)     76800       dropout_25[0][0]                 
___________________________________________________________________________________________

In [143]:
predicted = model1.predict(x_test_pad)
pred_class = np.argmax(predicted, axis=-1) 
from sklearn.metrics import hamming_loss, accuracy_score,precision_score
p1=pred_class.T
hm=[]
ac=[]
x1=[]
x2=[]
for i in range(len(y_test)):
  hm.append(hamming_loss(y_test[i], p1[i]))
print(np.mean(hm))
print (jacard(y_test, p1))

0.04333333333333334
0.7928333333333335


In [144]:
predicted = model.predict(x_test_pad)
pred_class = np.argmax(predicted, axis=-1) 
p=pred_class.T
hm=[]
ac=[]
x1=[]
x2=[]
for i in range(len(y_test)):
  hm.append(hamming_loss(y_test[i], p[i]))
print(np.mean(hm))
print (jacard(y_test, p))

0.04055555555555555
0.8033333333333336


In [145]:
predicted3 = model3.predict(x_test_pad)
pred_class = np.argmax(predicted3, axis=-1) 
p3=pred_class.T
hm=[]
ac=[]
x1=[]
x2=[]
for i in range(len(y_test)):
  hm.append(hamming_loss(y_test[i], p3[i]))
print(np.mean(hm))
print (jacard(y_test, p3))

0.04666666666666667
0.7767499999999999


In [146]:
from scipy import stats as s
from statistics import mode
final_pred = []
for i in range(0,len(x_test_pad)):
  x = s.mode([p1[i], p[i], p3[i]])
  final_pred.append(x[0])

In [147]:
hm=[]
for i in range(len(y_test)):
  hm.append(hamming_loss(y_test[i], final_pred[i][0]))
print(np.mean(hm))

0.03944444444444445


In [148]:
from sklearn.metrics import jaccard_score
ac=[]
for i in range(len(y_test)):
  ac.append(jaccard_score(y_test[i], final_pred[i][0],average='macro'))
print(np.mean(ac))

0.8764831349206349
